### Job assignment
Mark wishes to define the scheduling of costumers that he must attend in the next 3 days. The list of jobs (demands) with the duration of job and its profit is defined below:
- A) duration 2h, profit 200 USD
- B) duration 3h, profit 500 USD
- C) duration 5h, profit 300 USD
- D) duration 2h, profit 100 USD
- E) duration 6h, profit 1,000 USD
- F) duration 4h, profit 300 USD


Mark wants to maximize the profit for the next 3 days working 6h per day. Which demands he should attend per day?
- Neglect the traveling time
- Each demand only can be attended once
- Mark wish to do a maximum of 1 job per day

In [1]:
import pyomo.environ as pe
import pyomo.opt as po

In [2]:
m = pe.ConcreteModel()

m.I = pe.RangeSet(6)
m.T = pe.RangeSet(3)
m.x = pe.Var(m.I, m.T, domain=pe.Binary)

duration = {
    1: 2,
    2: 3,
    3: 5,
    4: 2,
    5: 6,
    6: 4
}
m.duration = pe.Param(m.I, initialize=duration)

profit = {
    1: 200,
    2: 500,
    3: 300,
    4: 100,
    5: 1000,
    6: 300
}
m.profit = pe.Param(m.I, initialize=profit)
m.max_hours = pe.Param(initialize=6)

In [3]:
def _six_hours(m, t):
    return sum(m.x[i, t]*m.duration[i] for i in m.I) <= m.max_hours
m.six_hours = pe.Constraint(m.T, rule=_six_hours)

def _one_job(m, i):
    return sum(m.x[i, t] for t in m.T) <= 1
m.one_job = pe.Constraint(m.I, rule=_one_job)

def _one_job_per_day(m, t):
    return sum(m.x[i, t] for i in m.I) <= 1
m.one_job_per_day = pe.Constraint(m.T, rule=_one_job_per_day)

m.obj = pe.Objective(
    expr=sum(sum(m.x[i,t] * m.profit[i] for i in m.I) for t in m.T),
    sense=pe.maximize)


solver = po.SolverFactory('glpk')
results = solver.solve(m)


In [4]:
m.x.display()
print(pe.value(m.obj))

x : Size=18, Index=x_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :     0 :   0.0 :     1 : False : False : Binary
    (1, 2) :     0 :   0.0 :     1 : False : False : Binary
    (1, 3) :     0 :   0.0 :     1 : False : False : Binary
    (2, 1) :     0 :   0.0 :     1 : False : False : Binary
    (2, 2) :     0 :   1.0 :     1 : False : False : Binary
    (2, 3) :     0 :   0.0 :     1 : False : False : Binary
    (3, 1) :     0 :   0.0 :     1 : False : False : Binary
    (3, 2) :     0 :   0.0 :     1 : False : False : Binary
    (3, 3) :     0 :   0.0 :     1 : False : False : Binary
    (4, 1) :     0 :   0.0 :     1 : False : False : Binary
    (4, 2) :     0 :   0.0 :     1 : False : False : Binary
    (4, 3) :     0 :   0.0 :     1 : False : False : Binary
    (5, 1) :     0 :   1.0 :     1 : False : False : Binary
    (5, 2) :     0 :   0.0 :     1 : False : False : Binary
    (5, 3) :     0 :   0.0 :     1 : False : False : Binary
    (6, 1) : 